<a href="https://colab.research.google.com/github/jsh1021902/ai_for_machine_learning/blob/main/week2/3_making_dataframes_from_api_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_02/

/content/drive/MyDrive/Hands-On-Data-Analysis-with-Pandas-2nd-edition/ch_02


### Get Data from API

In [ ]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

In [ ]:
api

'https://earthquake.usgs.gov/fdsnws/event/1/query'

In [ ]:
response.json

<bound method Response.json of <Response [200]>>

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [ ]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [ ]:
earthquake_json['metadata']

{'generated': 1694397284000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-08-11&endtime=2023-09-10',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 10936}

Each element in the JSON array `features` is a row of data for our dataframe.

In [ ]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [ ]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.76,
  'place': '9 km SW of Volcano, Hawaii',
  'time': 1694302879760,
  'updated': 1694303209750,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv73572952',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=hv73572952&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 48,
  'net': 'hv',
  'code': '73572952',
  'ids': ',hv73572952,',
  'sources': ',hv,',
  'types': ',origin,phase-data,',
  'nst': 16,
  'dmin': None,
  'rms': 0.109999999,
  'gap': 103,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.8 - 9 km SW of Volcano, Hawaii'},
 'geometry': {'type': 'Point',
  'coordinates': [-155.282165527344, 19.3753337860107, 1.12000000476837]},
 'id': 'hv73572952'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [ ]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]

In [ ]:
earthquake_properties_data

[{'mag': 1.76,
  'place': '9 km SW of Volcano, Hawaii',
  'time': 1694302879760,
  'updated': 1694303209750,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/hv73572952',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=hv73572952&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 48,
  'net': 'hv',
  'code': '73572952',
  'ids': ',hv73572952,',
  'sources': ',hv,',
  'types': ',origin,phase-data,',
  'nst': 16,
  'dmin': None,
  'rms': 0.109999999,
  'gap': 103,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.8 - 9 km SW of Volcano, Hawaii'},
 {'mag': -0.1,
  'place': '38 km NNW of Pahrump, Nevada',
  'time': 1694302462481,
  'updated': 1694302589537,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nn00865664',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nn00865664&format=geojson',
  'felt': None,
  

In [ ]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.76,"9 km SW of Volcano, Hawaii",1694302879760,1694303209750,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",hv73572952,",",hv,",",origin,phase-data,",16.0,NaN,0.1100,103.00,ml,earthquake,"M 1.8 - 9 km SW of Volcano, Hawaii"
1,-0.10,"38 km NNW of Pahrump, Nevada",1694302462481,1694302589537,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nn00865664,",",nn,",",origin,phase-data,",8.0,0.15200,0.0531,190.58,ml,earthquake,"M -0.1 - 38 km NNW of Pahrump, Nevada"
2,2.00,"60 km NW of Aleneva, Alaska",1694299816654,1694299901390,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak023bl4tqu9,",",ak,",",origin,phase-data,",NaN,NaN,0.3700,NaN,ml,earthquake,"M 2.0 - 60 km NW of Aleneva, Alaska"
3,0.80,"10 km ENE of Ocotillo Wells, CA",1694298886540,1694377505983,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ci39662066,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",17.0,0.05748,0.1300,63.00,ml,earthquake,"M 0.8 - 10 km ENE of Ocotillo Wells, CA"
4,1.70,"126 km S of McCarthy, Alaska",1694298727064,1694298882339,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak023bl4pv6p,",",ak,",",origin,phase-data,",NaN,NaN,0.5600,NaN,ml,earthquake,"M 1.7 - 126 km S of McCarthy, Alaska"


### (Optional) Write Data to CSV

In [ ]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>